In [5]:
import os
import time
import shutil
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
def wait_for_element_to_load(driver, xpath, timeout=15):
    """Wait for an element to load on the page."""
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )

def wait_for_element_to_be_interactable(driver, xpath, timeout=10):
    """Wait for an element to be present and visible."""
    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )

**LOGIN AND HOME PAGE**

In [ ]:
driver = webdriver.Edge()
driver.get("https://shop.alphacomm.com/")
wait_for_element_to_be_interactable(driver, f'//*[@id="login-email"]').send_keys("Somaan@xclusivetradinginc.com")
wait_for_element_to_be_interactable(driver, f'//*[@id="login-password"]').send_keys("Metro57398XTI$")
wait_for_element_to_be_interactable(driver, '//*[@id="LoginRegister.View"]/section/div[2]/div/div/form/fieldset/div[6]/button').click()

**MAIN PATH INITIALIZATION**

In [9]:
wait_for_element_to_be_interactable(driver, nagivation_buttons[0]).click()

In [ ]:
# navigation to shop by category
for button in nagivation_buttons:
    wait_for_element_to_be_interactable(driver, button).click()

    #select main body where all the options are
    main_body_xpath = '//*[@id="facet-browse"]/section/div[2]/div[2]'
    main_body = driver.find_element(By.XPATH, main_body_xpath)
    

    # Get all immediate child elements (e.g., divs)
    items = main_body.find_elements(By.TAG_NAME, 'a')

    # list of items.text 
    items_text = []
    for item in items:
        items_text.append(item.text)

    for i in range(len(items)):
        print(f"Clicking on item {items_text[i]}")
        driver.find_element(By.XPATH, f'//*[@id="facet-browse"]/section/div[2]/div[2]/div[{i+1}]').click()
        time.sleep(5)
        print("going back")
        driver.find_element(By.XPATH, '//*[@id="main-container"]/div[1]/div/ul/li[3]/a').click()
        print()
        time.sleep(5)

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Edge()
wait = WebDriverWait(driver, 15)

# Step 1: Load main page and get to main page
driver.get("https://shop.alphacomm.com/")

wait_for_element_to_be_interactable(driver, f'//*[@id="login-email"]').send_keys("Somaan@xclusivetradinginc.com")
wait_for_element_to_be_interactable(driver, f'//*[@id="login-password"]').send_keys("Metro57398XTI$")
wait_for_element_to_be_interactable(driver, '//*[@id="LoginRegister.View"]/section/div[2]/div/div/form/fieldset/div[6]/button').click()

shop_by_category = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[1]/a'
shop_by_headset = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[2]/a'
shop_by_brand = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[3]/a'

nagivation_buttons = [ shop_by_category, shop_by_headset, shop_by_brand ]

In [41]:
for item in range(len(nagivation_buttons)): 
    wait_for_element_to_be_interactable(driver, nagivation_buttons[item]).click()

    # Step 2: Find all clickable divs inside the main body
    main_body_xpath = '//*[@id="facet-browse"]/section/div[2]/div[2]'
    wait.until(EC.presence_of_element_located((By.XPATH, main_body_xpath)))
    main_body = driver.find_element(By.XPATH, main_body_xpath)

    clickable_divs = main_body.find_elements(By.XPATH, './div')

    # Step 3: Open each div in a new tab
    main_window = driver.current_window_handle
    for i in range(len(clickable_divs)):
        try:
            # Re-find the clickable divs to avoid stale elements
            main_body = driver.find_element(By.XPATH, main_body_xpath)
            clickable_divs = main_body.find_elements(By.XPATH, './div')

            # Use JavaScript to open link in new tab
            driver.execute_script("window.open(arguments[0].querySelector('a').href, '_blank');", clickable_divs[i])
            time.sleep(1)  # Tiny delay to avoid spamming too fast
        except Exception as e:
            print(f"Error opening div {i}: {e}")

    # Step 4: Now handle all tabs one by one
    all_tabs = driver.window_handles

    for tab in all_tabs:
        if tab == main_window:
            continue  # Skip the main page
        try:
            driver.switch_to.window(tab)

            # Wait for something on the new page
            # Replace 'your_target_xpath_here' with what you want to scrape
            your_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="facet-browse"]/section/div[2]/div[1]/section/div[2]/h1')))
            print(your_element.text)

            driver.close()  # Close tab after work done
        except Exception as e:
            print(f"Error in tab {tab}: {e}")

    # Step 5: Go back to main window
    driver.switch_to.window(main_window)

print("Done with all tabs!")


ATTACHABLES
SCREEN CARE
CHARGING
AUDIO
MEMORY
LG HANDSET
Error in tab 00C09DF526C8069088D815CCD17DE66F: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A6ABEE65+16773]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6A6A1F130+402672]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6A6D14BDA+1818314]
	(No symbol) [0x00007FF6A6857B86]
	(No symbol) [0x00007FF6A6857E6B]
	(No symbol) [0x00007FF6A6898C77]
	(No symbol) [0x00007FF6A6878F0F]
	(No symbol) [0x00007FF6A684DF8D]
	(No symbol) [0x00007FF6A6896658]
	(No symbol) [0x00007FF6A6878C33]
	(No symbol) [0x00007FF6A684D4D6]
	(No symbol) [0x00007FF6A684CA82]
	(No symbol) [0x00007FF6A684D303]
	(No symbol) [0x00007FF6A694060D]
	(No symbol) [0x00007FF6A694D642]
	(No symbol) [0x00007FF6A6945673]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6A6B99BAA+265882]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6A6A2C581+457025]
	Microsoft::Applications::Eve

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Edge()
wait = WebDriverWait(driver, 15)

# Step 1: Load main page and get to main page
driver.get("https://shop.alphacomm.com/")

wait_for_element_to_be_interactable(driver, f'//*[@id="login-email"]').send_keys("Somaan@xclusivetradinginc.com")
wait_for_element_to_be_interactable(driver, f'//*[@id="login-password"]').send_keys("Metro57398XTI$")
wait_for_element_to_be_interactable(driver, '//*[@id="LoginRegister.View"]/section/div[2]/div/div/form/fieldset/div[6]/button').click()

shop_by_category = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[1]/a'
shop_by_headset = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[2]/a'
shop_by_brand = '//*[@id="site-header"]/div[4]/div/div[2]/div/nav/ul/li[3]/a'

nagivation_buttons = [ shop_by_category, shop_by_headset, shop_by_brand ]

wait_for_element_to_be_interactable(driver, nagivation_buttons[0]).click()


main_body_xpath = '//*[@id="facet-browse"]/section/div[2]/div[2]'


In [22]:
# Function to wait for main body to appear
def wait_for_main_body():
    wait.until(EC.presence_of_element_located((By.XPATH, main_body_xpath)))
    return driver.find_element(By.XPATH, main_body_xpath)

# Step 2: Get main divs (Audio, Charging, Memory, etc.)
main_body = wait_for_main_body()
main_divs = main_body.find_elements(By.XPATH, './div')
main_window = driver.current_window_handle

# Loop over each main div
for i in range(len(main_divs)):
    try:
        # Re-fetch elements to avoid stale element error
        main_body = wait_for_main_body()
        main_divs = main_body.find_elements(By.XPATH, './div')

        # Click on main div
        main_divs[i].click()

        # Wait for sub-divs to appear inside same main body XPath
        sub_body = wait_for_main_body()
        sub_divs = sub_body.find_elements(By.XPATH, './div')

        # Now open each sub-div in a new tab
        for j in range(len(sub_divs)):
            try:
                sub_body = wait_for_main_body()
                sub_divs = sub_body.find_elements(By.XPATH, './div')

                # Open sub-div's link in a new tab
                driver.execute_script("window.open(arguments[0].querySelector('a').href, '_blank');", sub_divs[j])
                time.sleep(1)
            except Exception as e:
                print(f"Error opening sub-div {j}: {e}")

        # Now, switch to each new tab (except main_window) to scrape
        all_tabs = driver.window_handles
        for tab in all_tabs:
            if tab == main_window:
                continue
            try:
                driver.switch_to.window(tab)

                # Wait for target element on sub-div page
                your_target_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="facet-browse"]/section/div[2]/div[1]/section/div/h1')))
                print(your_target_element.text)

                driver.close()  # Close the tab after scraping
            except Exception as e:
                print(f"Error in sub-div tab: {e}")

        # After all sub-tabs closed, switch back to the main window
        driver.switch_to.window(main_window)

        # Now go back to main page
        driver.back()

        # Wait for main body to reload after going back
        wait_for_main_body()

        # SUPER IMPORTANT:
        # Re-find main divs fresh after going back
        main_body = wait_for_main_body()
        main_divs = main_body.find_elements(By.XPATH, './div')

    except Exception as e:
        print(f"Error processing main div {i}: {e}")

print("Finished scraping all divs and sub-divs!")


TRUE WIRELESS
OVER-EAR WIRELESS
IN-EAR WIRELESS
IN-EAR WIRED
AUDIO
Error opening sub-div 2: list index out of range
Error opening sub-div 3: list index out of range
Error opening sub-div 4: list index out of range
Error opening sub-div 5: list index out of range
IN-EAR WIRED
AUDIO ADAPTERS
Error opening sub-div 1: list index out of range
Error opening sub-div 2: list index out of range
Error opening sub-div 3: list index out of range
Error opening sub-div 4: list index out of range
AUDIO
Error processing main div 3: list index out of range
Error opening sub-div 1: Message: stale element reference: stale element not found in the current frame
  (Session info: MicrosoftEdge=135.0.3179.73); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A6ABEE65+16773]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6A6A1F130+402672]
	